AutoHorse Data

In [1]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
import pandas as pd
import seaborn as sns
from scipy.io import arff
from sklearn.model_selection import train_test_split

In [2]:
data = arff.loadarff('autoHorse.arff')
df = pd.DataFrame(data[0])
print(df.shape)

df.head()

(203, 66)


,symboling,normalized-losses,make=alfa-romero,make=audi,make=bmw,make=chevrolet,make=dodge,make=honda,make=isuzu,make=jaguar,...,fuel-system=idi,fuel-system=spdi,bore,stroke,compression-ratio,peak-rpm,city-mpg,highway-mpg,price,class
0,1.0,0.298429,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.664286,0.290476,0.1250,0.346939,0.222222,0.289474,0.207959,0.262500
1,1.0,0.298429,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.664286,0.290476,0.1250,0.346939,0.222222,0.289474,0.282558,0.262500
2,0.6,0.298429,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.100000,0.666667,0.1250,0.346939,0.166667,0.263158,0.282558,0.441667
3,0.8,0.518325,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.464286,0.633333,0.1875,0.551020,0.305556,0.368421,0.219254,0.225000
4,0.8,0.518325,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.464286,0.633333,0.0625,0.551020,0.138889,0.157895,0.306142,0.279167


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
symboling,203.0,0.566502,0.249477,0.0,0.400000,0.600000,0.800000,1.0
normalized-losses,203.0,0.298429,0.166688,0.0,0.188482,0.298429,0.376963,1.0
make=alfa-romero,203.0,0.014778,0.120963,0.0,0.000000,0.000000,0.000000,1.0
make=audi,203.0,0.034483,0.182917,0.0,0.000000,0.000000,0.000000,1.0
make=bmw,203.0,0.039409,0.195047,0.0,0.000000,0.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...
peak-rpm,203.0,0.398110,0.195647,0.0,0.265306,0.428571,0.551020,1.0
city-mpg,203.0,0.340038,0.182519,0.0,0.166667,0.305556,0.472222,1.0
highway-mpg,203.0,0.388125,0.182116,0.0,0.236842,0.368421,0.500000,1.0
price,203.0,0.201696,0.196100,0.0,0.066121,0.138523,0.282558,1.0


In [4]:
print(df["class"])

0      0.262500
1      0.262500
2      0.441667
3      0.225000
4      0.279167
         ...   
198    0.275000
199    0.466667
200    0.358333
201    0.241667
202    0.275000
Name: class, Length: 203, dtype: float64


In [5]:
X = df.drop("class",axis=1)
X.head()
features = (X.shape)[1] # feature value
print(features) 

65


In [6]:
Y = df['class']
Y.head()

0    0.262500
1    0.262500
2    0.441667
3    0.225000
4    0.279167
Name: class, dtype: float64

In [7]:
# Train - Test ayrımını yapmak lazım.
X_train_full,X_test,y_train_full,y_test = train_test_split(X,Y,test_size = 0.2)

In [8]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,test_size = 0.2)   

In [10]:
# Model
# 1 Hidden katmanlı yapı

model = Sequential()
model.add(Dense(33, activation='relu')) # Hidden layer
model.add(Dense(1)) # Output layer


In [11]:
# Adam optimizer ile MAE performans kriteri ile
model.compile(tf.keras.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])

In [12]:
history = model.fit(X_train,y_train, epochs=30, batch_size=64,validation_data=(X_valid,y_valid))

Epoch 1/30
3/3 [==============================] - 1s 200ms/step - loss: 0.2807 - accuracy: 0.0000e+00 - val_loss: 0.1689 - val_accuracy: 0.0303
Epoch 2/30
3/3 [==============================] - 0s 55ms/step - loss: 0.1886 - accuracy: 0.0000e+00 - val_loss: 0.1183 - val_accuracy: 0.0303
Epoch 3/30
3/3 [==============================] - 0s 34ms/step - loss: 0.1345 - accuracy: 0.0000e+00 - val_loss: 0.0922 - val_accuracy: 0.0303
Epoch 4/30
3/3 [==============================] - 0s 26ms/step - loss: 0.1014 - accuracy: 0.0000e+00 - val_loss: 0.0812 - val_accuracy: 0.0303
Epoch 5/30
3/3 [==============================] - 0s 22ms/step - loss: 0.0838 - accuracy: 0.0000e+00 - val_loss: 0.0746 - val_accuracy: 0.0303
Epoch 6/30
3/3 [==============================] - 0s 24ms/step - loss: 0.0731 - accuracy: 0.0000e+00 - val_loss: 0.0706 - val_accuracy: 0.0303
Epoch 7/30
3/3 [==============================] - 0s 27ms/step - loss: 0.0651 - accuracy: 0.0000e+00 - val_loss: 0.0664 - val_accuracy: 0.030

In [13]:
from scikeras.wrappers import KerasRegressor

In [14]:
keras_clf = KerasRegressor(model = model, optimizer="adam", epochs=30,batch_size=64,loss='mean_squared_error',verbose=0)

In [15]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor

In [16]:
bag_reg = BaggingRegressor(base_estimator=keras_clf,n_estimators=10,random_state=1)

In [17]:
ada_reg = AdaBoostRegressor(base_estimator=keras_clf,n_estimators=10,random_state=1)

In [18]:
ranSub_reg = BaggingRegressor(base_estimator=keras_clf,n_estimators=10,bootstrap=False,bootstrap_features=True,random_state=1)

In [19]:
from mlxtend.evaluate import paired_ttest_5x2cv

In [20]:
t1, p1 = paired_ttest_5x2cv(estimator1=bag_reg,
                              estimator2=ada_reg,
                              X=X, y=Y,
                              random_seed=1)
                            
if p1<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp9_8rtxd6\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpq5ryhjdn\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp6l7e36fa\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp6p9u0im2\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpt0lvt70j\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpqccnflza\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpc0xq_p3y\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp5ongzzq2\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpls6qtlix\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp4311twtz\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmphkbjtmpc\assets
INFO:tensorflow:Assets written t

In [26]:
t2, p2 = paired_ttest_5x2cv(estimator1=bag_reg,
                              estimator2=ranSub_reg,
                              X=X, y=Y,
                              random_seed=1)
                            
if p2<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpl4waflzf\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp6x8ga6k5\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpqprafdpz\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpo6a1u2fd\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp_ls2uy8l\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpwzuigffq\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp2ax666u4\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmprcm90tyi\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmptyif_otb\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpmamjvccj\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpww60e0y9\assets
INFO:tensorflow:Assets written t

In [22]:
t3, p3 = paired_ttest_5x2cv(estimator1=bag_reg,
                              estimator2=model,
                              X=X, y=Y,
                              random_seed=1)
                            
if p3<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp8m1qdjz5\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp_tfcdx93\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpohrj032p\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpwp1vi16m\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp9prpxez5\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpyojgqayf\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpuaob1pw2\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpggpyj2i7\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmptiumjd7x\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpp0_pr4ie\assets
4/4 [==============================] - 0s 3ms/step
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\

In [23]:
t4, p4 = paired_ttest_5x2cv(estimator1=ada_reg,
                              estimator2=ranSub_reg,
                              X=X, y=Y,
                              random_seed=1)
                            
if p4<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpi6_ix587\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp_2e4zce9\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmppl5z1xms\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpppd0qmct\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpz76djg3l\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp7k7q23_5\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpwzt3m9y8\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpyoj7hv0s\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpj3l9_exj\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpuvn6dr48\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmps2xipx7w\assets
INFO:tensorflow:Assets written t

In [24]:
t5, p5 = paired_ttest_5x2cv(estimator1=ada_reg,
                              estimator2=model,
                              X=X, y=Y,
                              random_seed=1)
                            
if p5<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpj9kl699l\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpmogwazy9\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpqwm5ucum\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp44avo_yy\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmphbfeoe4r\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmphpd1e1ne\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp95n356or\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpyafm47ic\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmprxpc4atb\assets
4/4 [==============================] - 0s 3ms/step
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp5l8pxsmv\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\

In [25]:
t6, p6 = paired_ttest_5x2cv(estimator1=ranSub_reg,
                              estimator2=model,
                              X=X, y=Y,
                              random_seed=1)
                            
if p6<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpc2_gb178\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpdaevup0r\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpitcr4az0\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpltp3i88u\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp4ke12nxe\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpz6tyawgq\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp2pxh0bfg\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp23awpjc4\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpujm5i6fn\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpbpp8xkaz\assets
4/4 [==============================] - 0s 6ms/step
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\